In [1]:
from pyspark.sql import SparkSession
from datetime import datetime, timedelta
from pyspark.sql.functions import *

In [11]:

spark = SparkSession.builder.appName('loadNodesData').getOrCreate()
airports_file = '/Users/ybatash/PycharmProjects/jupyter/tau-network-science/data/airports.csv'
adf = spark.read.csv(path=airports_file, header=True)
adf.createOrReplaceTempView('ADF')
adf.printSchema()


root
 |-- id: string (nullable = true)
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- latitude_deg: string (nullable = true)
 |-- longitude_deg: string (nullable = true)
 |-- elevation_ft: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- scheduled_service: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- home_link: string (nullable = true)
 |-- wikipedia_link: string (nullable = true)
 |-- keywords: string (nullable = true)



In [14]:
adf.show(100,False)

+------+-----+-------------+------------------------------------------------+------------------+-------------------+------------+---------+-----------+----------+-----------------+-----------------+--------+---------+----------+------------------------------------------------------------------------------+-----------------------------------------------------------+--------------------------+
|id    |ident|type         |name                                            |latitude_deg      |longitude_deg      |elevation_ft|continent|iso_country|iso_region|municipality     |scheduled_service|gps_code|iata_code|local_code|home_link                                                                     |wikipedia_link                                             |keywords                  |
+------+-----+-------------+------------------------------------------------+------------------+-------------------+------------+---------+-----------+----------+-----------------+-----------------+--------+---

In [3]:
# spark.sql("""
# select  count(*) as row_count,
#         sum( case when id                is not null and id                <>'' and id                <> 'NA' then 1 else 0 end)/count(*) * 100 as id,
#         sum( case when ident             is not null and ident             <>'' and ident             <> 'NA' then 1 else 0 end)/count(*) * 100 as ident,
#         sum( case when type              is not null and type              <>'' and type              <> 'NA' then 1 else 0 end)/count(*) * 100 as airport_type,
#         sum( case when name              is not null and name              <>'' and name              <> 'NA' then 1 else 0 end)/count(*) * 100 as airport_name,
#         sum( case when latitude_deg      is not null and latitude_deg      <>'' and latitude_deg      <> 'NA' then 1 else 0 end)/count(*) * 100 as latitude_deg,
#         sum( case when longitude_deg     is not null and longitude_deg     <>'' and longitude_deg     <> 'NA' then 1 else 0 end)/count(*) * 100 as longitude_deg,
#         sum( case when elevation_ft      is not null and elevation_ft      <>'' and elevation_ft      <> 'NA' then 1 else 0 end)/count(*) * 100 as elevation_ft,
#         sum( case when continent         is not null and continent         <>'' and continent         <> 'NA' then 1 else 0 end)/count(*) * 100 as continent,
#         sum( case when iso_country       is not null and iso_country       <>'' and iso_country       <> 'NA' then 1 else 0 end)/count(*) * 100 as iso_country,
#         sum( case when iso_region        is not null and iso_region        <>'' and iso_region        <> 'NA' then 1 else 0 end)/count(*) * 100 as iso_region,
#         sum( case when municipality      is not null and municipality      <>'' and municipality      <> 'NA' then 1 else 0 end)/count(*) * 100 as municipality,
#         sum( case when scheduled_service is not null and scheduled_service <>'' and scheduled_service <> 'NA' then 1 else 0 end)/count(*) * 100 as scheduled_service,
#         sum( case when gps_code          is not null and gps_code          <>'' and gps_code          <> 'NA' then 1 else 0 end)/count(*) * 100 as gps_code,
#         sum( case when iata_code         is not null and iata_code         <>'' and iata_code         <> 'NA' then 1 else 0 end)/count(*) * 100 as iata_code,
#         sum( case when local_code        is not null and local_code        <>'' and local_code        <> 'NA' then 1 else 0 end)/count(*) * 100 as local_code,
#         sum( case when home_link         is not null and home_link         <>'' and home_link         <> 'NA' then 1 else 0 end)/count(*) * 100 as home_link,
#         sum( case when wikipedia_link    is not null and wikipedia_link    <>'' and wikipedia_link    <> 'NA' then 1 else 0 end)/count(*) * 100 as wikipedia_link,
#         sum( case when keywords          is not null and keywords          <>'' and keywords          <> 'NA' then 1 else 0 end)/count(*) * 100 as keywords
# from    ADF

# """).show(10, False)


In [3]:
nodes_df = spark.sql("""
select  ident             as Id,
        name              as Label,
        type              as Airport_Type,
        iso_country       as Country,
        latitude_deg      as lat,
        longitude_deg     as lng
from    ADF joint eFilter on 
""")
# where   iso_country = 'IL'


nodes_df.createOrReplaceTempView('ADF')

In [6]:
# SMALL Nodes FILE

edges_file = '/Users/ybatash/PycharmProjects/jupyter/tau-network-science/b_01_2019/edges_2019_01.csv'
edf = spark.read.csv(path=edges_file, header=True)

edf.count()

136897

In [7]:
edf.createOrReplaceTempView('EDF')
edf.printSchema()
edf.count()


root
 |-- Source: string (nullable = true)
 |-- Target: string (nullable = true)
 |-- Weight: string (nullable = true)



136897

In [15]:
edges_filter_df = spark.sql("""select distinct Source as Id from EDF union select distinct Target from EDF""")


edges_filter_df.createOrReplaceTempView('eFilter')

edges_filter_df.count()

10464

In [17]:
nodes_df = spark.sql("""
select  A.ident             as Id,
        A.name              as Label,
        A.type              as Airport_Type,
        A.iso_country       as Country,
        A.latitude_deg      as lat,
        A.longitude_deg     as lng
from    ADF as A join eFilter as E 
        on A.ident=E.Id
""")
# where   iso_country = 'IL'

nodes_df.count()


10443

In [19]:
nodes_df.createOrReplaceTempView('N')

spark.sql("""select count(*) from N where Airport_Type in ('medium_airport','large_airport')""").show(10,False)

+--------+
|count(1)|
+--------+
|1530    |
+--------+



In [18]:
nodes_df.repartition(1).write.format("com.databricks.spark.csv").option("header", "true").save("new_nodes")


In [30]:
# flightlist_file = '/Users/ybatash/PycharmProjects/jupyter/tau-network-science/data/raw_data/flightlist_*.csv.gz'
# fdf = spark.read.csv(path=flightlist_file, header=True)
# fdf = fdf.withColumn("filename", input_file_name())
# fdf.createOrReplaceTempView('FDF')
# fdf.printSchema()


In [31]:
# spark.sql("""
# select 
#         filename,
#         count(*) as row_count,
#         sum( case when callsign     is not null then 1 else 0 end)/count(*) * 100 as nn_count_callsign     ,
#         sum( case when number       is not null then 1 else 0 end)/count(*) * 100 as nn_count_number       ,
#         sum( case when icao24       is not null then 1 else 0 end)/count(*) * 100 as nn_count_icao24       ,
#         sum( case when registration is not null then 1 else 0 end)/count(*) * 100 as nn_count_registration ,
#         sum( case when typecode     is not null then 1 else 0 end)/count(*) * 100 as nn_count_typecode     ,
#         sum( case when origin       is not null then 1 else 0 end)/count(*) * 100 as nn_count_origin       ,
#         sum( case when destination  is not null then 1 else 0 end)/count(*) * 100 as nn_count_destination  ,

#        sum( case
#                   when origin       is not null and
#                        destination  is not null
#                     then 1 else 0 end)/count(*) * 100 as nn_count_origin_destination       ,


#         sum( case when firstseen    is not null then 1 else 0 end)/count(*) * 100 as nn_count_firstseen    ,
#         sum( case when lastseen     is not null then 1 else 0 end)/count(*) * 100 as nn_count_lastseen     ,
#         sum( case when day          is not null then 1 else 0 end)/count(*) * 100 as nn_count_day          ,
#         sum( case when latitude_1   is not null then 1 else 0 end)/count(*) * 100 as nn_count_latitude_1   ,
#         sum( case when longitude_1  is not null then 1 else 0 end)/count(*) * 100 as nn_count_longitude_1  ,
#         sum( case when altitude_1   is not null then 1 else 0 end)/count(*) * 100 as nn_count_altitude_1   ,
#         sum( case when latitude_2   is not null then 1 else 0 end)/count(*) * 100 as nn_count_latitude_2   ,
#         sum( case when longitude_2  is not null then 1 else 0 end)/count(*) * 100 as nn_count_longitude_2  ,
#         sum( case when altitude_2   is not null then 1 else 0 end)/count(*) * 100 as nn_count_altitude_2
# from FDF
# where origin is not null and destination  is not null
# group by 1
# """).show(10, False)


In [32]:
# res_df = spark.sql("""
# select origin                   as origin_airport,
#        destination              as destination_airport,
#        count(*)                 as flights_count 
# from   FDF     
# where   1=1
#         and origin is not null and destination  is not null
#         and filename like 'file:///Users/ybatash/PycharmProjects/jupyter/tau-network-science/data/raw_data/flightlist_2019_%.csv.gz'
# group by 1,2        
# """)
# res_df.createOrReplaceTempView('FDF')

In [25]:
# spark.sql("""
# with T 
# as(
#     select F.origin_airport      as origin_airport,
#            A.airport_id          as airport_or,  
#            F.destination_airport as destination_airport,
#            B.airport_id          as airport_des,
#            F.flights_count       as flights_count       
#     from   FDF F 
#            Left Join ADF A
#            on F.origin_airport = A.airport_id
#            Left Join ADF B
#            on F.destination_airport = B.airport_id)

# select count(distinct coalesce(origin_airport,destination_airport)) 

# from T
# where  1=1
#        and T.airport_or is null or T.airport_des is null

# """).show(10, False)


+-------------------------------------------------------------+
|count(DISTINCT coalesce(origin_airport, destination_airport))|
+-------------------------------------------------------------+
|1074                                                         |
+-------------------------------------------------------------+

